### Librerias

In [1]:
import pandas as pd
import numpy as np
import sys
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, TimestampType, FloatType, DateType
from pyspark.sql.functions import lit, col, trim, regexp_replace, regexp_extract, when, concat
from datetime import datetime, timedelta
import argparse
import json
from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
import pyspark.sql.functions as F
import random

StatementMeta(sparkpooldev01, 5579, 2, Finished, Available)

In [2]:
# Nombre del Key Vault
key_vault = ''

# Nombre de los secretos cuentas de almacenamiento y key1
stccrdv001Name = ''
stccrdv001key = ''

StatementMeta(sparkpooldev01, 5579, 3, Finished, Available)

## Credenciales

In [3]:
# Definimos los secretos del Key Vault en nuestras variables. Nombre de los secretos cuentas de almacenamiento y key1
synapseusername_stccrdv001 = TokenLibrary.getSecret(key_vault, stccrdv001Name)  
synapsepassword_stccrdv001 = TokenLibrary.getSecret(key_vault, stccrdv001key)

StatementMeta(sparkpooldev01, 5579, 4, Finished, Available)

In [4]:
# Define el nombre del contenedor
adls_container_name = ""

# Define la duración y los permisos del token de acceso SAS
sas_expiry = datetime.utcnow() + timedelta(hours=1)
sas_permissions = ContainerSasPermissions(read=True, list=True)  # Puedes ajustar los permisos según tus necesidades

# Genera el token de acceso SAS
sas_token = generate_container_sas(
    account_name=synapseusername_stccrdv001,
    container_name=adls_container_name,
    account_key=synapsepassword_stccrdv001,
    expiry=sas_expiry,
    permission=sas_permissions
)

# Configura Spark para utilizar el token de acceso SAS al acceder a Azure Data Lake Storage Gen2
spark.conf.set(
    'fs.azure.sas.%s.%s.dfs.core.windows.net' % (adls_container_name, synapseusername_stccrdv001),
    sas_token)

StatementMeta(sparkpooldev01, 5579, 5, Finished, Available)

In [5]:
spark.conf.get("spark.sql.adaptive.enabled", "true")
spark.conf.get("spark.sql.adaptive.coalescePartitions", "true")

StatementMeta(sparkpooldev01, 5579, 6, Finished, Available)

'true'

# **Carga de archivos**

In [6]:
# Iniciar una sesión de Spark
spark = SparkSession.builder.getOrCreate()
 
# Datos de los departamentos
departments_data = [(1, "Sales"), (2, "Marketing"), (3, "IT"), (4, "Human Resources"), (5, "Finance"), (6, "Operations")]
departments_df = spark.createDataFrame(departments_data, ["department_id", "department_name"])
 
# Datos de los puestos de trabajo
positions_data = [(101, "Manager"), (102, "Analyst"), (103, "Developer"), (104, "HR Specialist"), (105, "Accountant"), (106, "Data Engineer")]
positions_df = spark.createDataFrame(positions_data, ["position_id", "position_name"])
 
# Datos de los empleados
random.seed(0)
employees_data = [(i, f"Employee{i}", random.choice([1, 2, 3, 4, 5]), random.choice([101, 102, 103, 104, 105]), f"2021-{random.randint(1, 12)}-{random.randint(1, 28)}") for i in range(1, 101)]
employees_df = spark.createDataFrame(employees_data, ["employee_id", "employee_name", "department_id", "position_id", "hire_date"])

StatementMeta(sparkpooldev01, 5579, 7, Finished, Available)

In [7]:
display(departments_df)

StatementMeta(sparkpooldev01, 5579, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3be45766-3093-49b1-9b35-3c8a2fffdaef)

In [8]:
display(positions_df)

StatementMeta(sparkpooldev01, 5579, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7f2a8141-9017-4296-a02f-c0e764515e33)

In [9]:
display(employees_df)

StatementMeta(sparkpooldev01, 5579, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, a8c4d033-d525-4a0d-8ad6-d1186070dd53)

In [10]:
departments = f"df_departments.parquet"
employees = f"df_employees.parquet"
positions = f"df_positions.parquet"

StatementMeta(sparkpooldev01, 5579, 11, Finished, Available)

In [11]:
departments_df = departments_df.toPandas()
departments_df.to_parquet(departments)

employees_df = employees_df.toPandas()
employees_df.to_parquet(employees)

positions_df = positions_df.toPandas()
positions_df.to_parquet(positions)


StatementMeta(sparkpooldev01, 5579, 12, Finished, Available)

## Enviar a SQL

In [14]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Conecta con la cuenta de almacenamiento Blob
blob_service_client = BlobServiceClient(account_url=f".blob.core.windows.net", credential=synapsepassword_stccrdv001)

# Crea un cliente para el contenedor en el que deseas guardar el archivo
container_name = "adlsdlccrdv001"
container_client = blob_service_client.get_container_client(container_name)

# Ruta en Blob Storage
ruta_blob = f"reject/{positions}"

# Comprobar si el archivo ya existe y eliminarlo si es necesario
blob_client = container_client.get_blob_client(ruta_blob)
if blob_client.exists():
    blob_client.delete_blob()

# Sube el archivo al contenedor
blob_client = container_client.get_blob_client(ruta_blob)
with open(positions, "rb") as data:
    blob_client.upload_blob(data)

StatementMeta(sparkpooldev01, 5579, 15, Finished, Available)